# Process Concaténation model

In [4]:
import torch
from Snt import Snt
from Matrice import Matrice
import Utils_data
import Utils_concat
import importlib
importlib.reload(Utils_data)
importlib.reload(Utils_concat)
_FULL_SNT = False # permet prendre en compte la phrase courante ou non
id = 1850

for id in range(188, 189):
    print(f"[DEBUG] id: {id}")
    r_path=f"/home/getalp/lopezfab/lig/temp/temp/test_attn/{id}.json"
    data=Utils_data.lecture_data(r_path)
    _OUTPUT_PATH=f"/home/getalp/lopezfab/Documents/concat/{id}"

    # Traitement des Phrases extraites
    ssl = data['src_segments_labels']
    src = Snt(identifiant=-1, tokens=Utils_concat.ajoute_eos_tokens_src(_snt=data["src_tokens"].split(), src_segments_labels=ssl))
    src_cutted = Utils_concat.full_sentence_to_ctx_and_crt(src)

    # Au moins une phrase de contexte et la phrase courante (+ 1 car contient une phrase vide quand le nb de contexte est inférieur à la normale)
    if len(src_cutted) > 2: 
        # Extraction des phrases de contexte et de la phrase courante
        ctxs = []
        for k in range(len(src_cutted[:-1])):
            ctxs.append(Snt(identifiant=int(data["id"]) - len(src_cutted[k:-1]), tokens = src_cutted[k]))
        # print(f"[debug] len(full_ctx): {len(full_ctx)}")

        # Extraction des différentes matrices à travers les 6 layers et les 8 têtes d'attention de chaque layer
        layers = []
        for layer in range(len(data['heads_enc_attn'])): # Pour chaque layer
            heads = []
            for head in range(len(data['heads_enc_attn'][layer])): # on extrait chaque tête par layer
                full_matrice = torch.tensor(data['heads_enc_attn'][layer][head])
                full_matrice = full_matrice.squeeze() # on supprime une dimension qui semble inutile (=1)
                heads.append(Matrice(full_matrice))
            layers.append(heads)
        # layers : L x [nb_heads x [torch.Tensor(N x N)]]

        # Traitement du cas particulier où un contexte n'est pas présent. Suppression des reliquats dans le contexte et les matrices d'attention
        for i in range(len(ctxs)-1, -1, -1):
            if ctxs[i].tokens == ["<eos>"]:
                del ctxs[i]
                del src_cutted[i]
                for layer in range(len(layers)):
                    for head in range(len(layers[layer])):
                        layers[layer][head].matrice = torch.cat([layers[layer][head].matrice[1:, 1:]])

        # Récupération de l'ensemble des tokens
        full_ctx = Snt(identifiant= id - len(ctxs), tokens= ctxs[0].tokens)
        if len(ctxs) > 1:
            for snt in ctxs[1:]:
                full_ctx.tokens += snt.tokens
                # print(f"[debug] len(full_ctx): {len(full_ctx)}")
        crt = Snt(identifiant=data['id'], tokens= src_cutted[-1])
        if _FULL_SNT: # permet prendre en compte la phrase courante ou non
            full_ctx.tokens += crt.tokens

        # print((f"[DEBUG] taille des matrices: {layers[0][0].matrice.size()}"))
        # print(f"[DEBUG] taille des phrases complète vs tailles respectives: {sum([len(ctx) for ctx in ctxs] + [len(crt)])} vs. [{[len(ctx) for ctx in ctxs]}, {len(crt)}]")

        # Pour chaque layer, pour chaque tête d'attention, on découpe la matrice en combinaison de k3*k3, k3*k2... k2*k3, k2*k2,... crt*crt
        for layer in range(len(layers)):
            for head in range(len(layers[layer])):
                full_matrice_cutted = Utils_concat.cut_matrix_into_sentences(layers[layer][head], src_cutted)
                # Dernière liste correspond à la phrase courante vers les phrases de contexte et la phrase courante
                # _FULL_SNT permet prendre en compte la phrase courante ou non
                if _FULL_SNT:
                    layers[layer][head].matrice = torch.cat([matrice.matrice for matrice in full_matrice_cutted[-1][:]], dim = 1)
                else:
                    layers[layer][head].matrice = torch.cat([matrice.matrice for matrice in full_matrice_cutted[-1][:-1]], dim = 1)
        # print(f"[DEBUG] taille de la matrice découpée: {[matrice.matrice.size() for matrice in full_matrice_cutted[-1]]}")
        # print(f"[DEBUG] taille de la matrice reconstituée: {layers[layer][head].matrice.size()}")
        # print(f"[DEBUG] élément matrice: {layers[0][0].matrice[-2, ...]}")
        for layer in range(len(layers)):
            for head in range(len(layers[layer])):
                layers[layer][head].suppr_inf()
                layers[layer][head].norm_tensor()
                layers[layer][head].ecriture_xslx(crt=crt, ctx=full_ctx, precision= 4, absolute_folder=f"{_OUTPUT_PATH}/full_matrice/{layer}", filename=f"{head}", create_folder_path=True)


[DEBUG] id: 188
[['<eos>'], ['So', 'over', 'the', 'long', 'course', 'of', 'human', 'history', ',', 'the', 'infec@@', 'tious', 'disease', 'that', '&apos;s', 'killed', 'more', 'humans', 'than', 'any', 'other', 'is', 'malaria', '.', '<eos>'], ['It', '&apos;s', 'carried', 'in', 'the', 'bit@@', 'es', 'of', 'infected', 'mo@@', 'squ@@', 'it@@', 'os', ',', 'and', 'it', '&apos;s', 'probably', 'our', 'oldest', 's@@', 'cour@@', 'ge', '.', '<END>']]


# Test MultiEnc

In [1]:
import Utils_data as ud
import Utils
import torch
import Matrice
import Sl_matrice
from Snt import Snt

precision = 8


for id in range(1850, 1851):
    print(f"sentence : {id}")
    precision = 8
    r_path=f"/home/getalp/lopezfab/lig/temp/temp/temp/han_attn2/{id}.json"
    OUTPUT_PATH = f"/home/getalp/lopezfab/Documents/{id}"

    # Lecture des données
    data=ud.lecture_data(r_path)
    crt, ctxs, ctxs_heads, sl_heads = ud.lecture_multi_enc_objet(data)

    # Corrections des données dans les cas où il y a moins de 3 contextes
    mask = torch.ones(sl_heads[0].matrice.shape[1], dtype = torch.bool)
    for k in range(len(ctxs)-1, -1, -1):
        # On supprime les contextes inutiles
        if len(ctxs[k].tokens) == 1 or (len(ctxs[k].tokens) > 1 and ctxs[k].tokens[-2] == "<pad>"):
            del ctxs[k]
            del ctxs_heads[k]
            mask[k] = False
        else:
            # On corrige un problème de padding qui apparait quand il y a moins de 3 contextes
            for h in range(len(ctxs_heads[0])):
                ctxs_heads[k][h].matrice = ctxs_heads[k][h].matrice[..., -len(ctxs[k]):]

    # S'il y a au moins une phrase de contexte on l'a traite
    if len(ctxs) >= 1:
        for h in range(len(sl_heads)):
            sl_heads[h].matrice = sl_heads[h].matrice[:, mask]
        
        # Process de la phrase courante
        print(f"[après] {crt}")
        list_crt_suppr_pad = crt.suppr_pad(strict=True)
        print(f"[avant] {crt}")
        print(list_crt_suppr_pad)
        list_crt_fusion_bpe = crt.fusion_bpe()

        mean_ctxs_heads = []
        # Traitement de chaque contexte
        for k in range(len(ctxs)):
            # Process de la phrases de contexte K
            list_ctx_suppr_pad = ctxs[k].suppr_pad(strict=True)
            list_ctx_fusion_bpe = ctxs[k].fusion_bpe()

            for head in range(len(ctxs_heads[k])):
                # Traitement de chaque tête d'attention entre la phrase courante et chaque phrase de contexte k
                ctxs_heads[k][head].suppr_pad(row_list_suppr_pad= list_crt_suppr_pad, col_list_suppr_pad=  list_ctx_suppr_pad)
                ctxs_heads[k][head].fusion_bpe(row_list_fusion_bpe= list_crt_fusion_bpe, col_list_fusion_bpe= list_ctx_fusion_bpe)
                ctxs_heads[k][head].suppr_inf()

            # Pour chaque phrase de contexte, on récupère la moyenne des poids d'attention 
            # de la phrase courante vers la phrase de contexte (doit être effectuée avant la normalisation )
            mean_ctxs_heads.append(Matrice.Matrice(Utils.mean_matrices([ctxs_heads[k][head].matrice for head in range(len(ctxs_heads[k])) ])))
            mean_ctxs_heads[k].norm_tensor() # On peut normaliser la moyenne car on ne l'utilise pas dans la contextualisation

        for sl_head in range(len(sl_heads)):
            # Traitement de chaque tête d'attention entre la phrase courante et l'ensemble des phrases de contexte
            sl_heads[sl_head].suppr_pad(row_list_suppr_pad= list_crt_suppr_pad)
            sl_heads[sl_head].fusion_bpe(row_list_fusion_bpe= list_crt_fusion_bpe, action= "mean")
            

        # On récupère la moyenne des têtes d'attention du mécanisme sentence level
        mean_sl_heads = Sl_matrice.Sl_matrice(Utils.mean_matrices([sl_heads[sl_head].matrice for sl_head in range(len(sl_heads))]))
        mean_sl_heads.norm_tensor() # On peut normaliser la moyenne car on l'utilise pas dans la contextualisation

        
        # Contextualisation entre le mécanisme d'attention word-level et le mécanisme d'attention sentence-level
        # Récupération de l'ensemble des phrases de contexte en une seule d'identifiant -1
        full_ctx = Snt(identifiant=crt.identifiant, tokens=ctxs[0].tokens) if len(ctxs)>= 1 else None
        if full_ctx is not None:
            for k in range(1, len(ctxs)):
                full_ctx.tokens += ctxs[k].tokens

        # écriture Token-level K x H x crt x ctxs[k] + means
        for k in range(len(ctxs)):
            # Pour chaque phrase de contexte K, 
            # On écrit les matrices de chaque heads entre la phrase  courante et la phrase de contexte K
            # Puis la moyenne de têtes
            for head in range(len(ctxs_heads[k])):
                ctxs_heads[k][head].ecriture_xslx(crt= crt, 
                                                    ctx= ctxs[k],
                                                    absolute_folder= f"{OUTPUT_PATH}/token_level/{head}", 
                                                    filename=f"ctx_{k}", 
                                                    precision=precision, 
                                                    create_folder_path=True)
            mean_ctxs_heads[k].ecriture_xslx(crt= crt,
                                                ctx= ctxs[k],
                                                absolute_folder= f"{OUTPUT_PATH}/token_level", 
                                                filename=f"mean_ctx_{k}", 
                                                create_folder_path=True)

        # écriture Sentence-level H x crt x nb_ctx + means
        for sl_head in range(len(sl_heads)):
            # Pour chaque tête d'attention sentence-level,
            # On écrit la tête d'attention entre la phrase courante et les K phrases
            # Puis on écrit la moyenne des têtes d'attention
            # TODO: à vérifier si sl_heads est en mode k3 x k2 x k1 ou k1 x k2 x k3
            # Si sl_heads est en mode  k1 x k2 x k3 alors .flip le passe en mode k3 x k2 x k1 pour plus de lisibilité
            sl_heads[sl_head].matrice=sl_heads[sl_head].matrice.flip(1)
            sl_heads[sl_head].ecriture_xslx(crt= crt, 
                                                absolute_folder= f"{OUTPUT_PATH}/sentence_level", 
                                                filename=f"sl_head_{sl_head}", 
                                                precision=precision, 
                                                create_folder_path=True)
        mean_sl_heads.ecriture_xslx(crt= crt,
                                        absolute_folder= f"{OUTPUT_PATH}/sentence_level", 
                                        filename=f"mean", 
                                        precision=precision, 
                                        create_folder_path=True)


        print(full_ctx)
        # Traitement et écriture des phrases contextualisé crt x nb_ctx
        for h_sl in range(len(sl_heads)):
            for h_tl in range(len(ctxs_heads[0])):
                
                temp = sl_heads[h_sl].contextualise_matrice([ctxs_heads[k][h_tl] for k in range(sl_heads[h_sl].matrice.size(1))])
                temp.ecriture_xslx(crt= crt,
                                            ctx= full_ctx,
                                            absolute_folder= f"{OUTPUT_PATH}/full_matrice/sl_{h_sl}", 
                                            filename=f"head_{h_tl}", 
                                            precision=8,
                                            create_folder_path=True)


## OUTPUT
# OUTPUT_PATH/{id}/
#  - sentence_level
#     - sl_heads_{head}.xslx
#        ...
#  - token_level
#     - {head}
#        - ctx_{k}.xslx
#           ...
#  - full_matrice
#     - sl_{sl_head}
#        - head_{head}.xslt
#           ...
#        ...

sentence : 1850
dict_keys(['id', 'crt', 'ctxs', 'matrices', 'SL_matrice', 'heads'])
[après] Snt(id=1850, tokens=['<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', 'In', 'their', 'home', 'countries', ',', 'they', 'experienced', 'sexual', 'violence', ',', 'forced', 'marriage', ',', 'honour', 'k@@', 'ill@@', 'ings', ',', '